In [1]:
from __future__ import print_function
import os
import six

import numpy as np
import paddle
import paddle.fluid as fluid
import paddle.fluid.layers as layers

dict_size = 30000  # 词典大小
bos_id = 0  # 词典中start token对应的id
eos_id = 1  # 词典中end token对应的id
source_dict_size = target_dict_size = dict_size  # 源/目标语言字典大小
word_dim = 512  # 词向量维度
hidden_dim = 512  # 编码器中的隐层大小
decoder_size = hidden_dim  # 解码器中的隐层大小
max_length = 256  # 解码生成句子的最大长度
beam_size = 4  # beam search的柱宽度
batch_size = 64  # batch 中的样本数

model_save_dir = "machine_translation.inference.model"

In [2]:
def data_func(is_train=True):
    # 源语言source数据
    src = fluid.data(name="src", shape=[None, None], dtype="int64")
    src_sequence_length = fluid.data(name="src_sequence_length",
                                     shape=[None],
                                     dtype="int64")
    inputs = [src, src_sequence_length]
    # 训练时还需要目标语言target和label数据
    if is_train:
        trg = fluid.data(name="trg", shape=[None, None], dtype="int64")
        trg_sequence_length = fluid.data(name="trg_sequence_length",
                                        shape=[None],
                                        dtype="int64")
        label = fluid.data(name="label", shape=[None, None], dtype="int64")
        inputs += [trg, trg_sequence_length, label]
    # data loader
    loader = fluid.io.DataLoader.from_generator(feed_list=inputs,
                                                capacity=10,
                                                iterable=True,
                                                use_double_buffer=True)
    return inputs, loader

In [3]:
def encoder(src_embedding, src_sequence_length):
    # 使用GRUCell构建前向RNN
    encoder_fwd_cell = layers.GRUCell(hidden_size=hidden_dim)
    encoder_fwd_output, fwd_state = layers.rnn(
        cell=encoder_fwd_cell,
        inputs=src_embedding,
        sequence_length=src_sequence_length,
        time_major=False,
        is_reverse=False)
    # 使用GRUCell构建反向RNN
    encoder_bwd_cell = layers.GRUCell(hidden_size=hidden_dim)
    encoder_bwd_output, bwd_state = layers.rnn(
        cell=encoder_bwd_cell,
        inputs=src_embedding,
        sequence_length=src_sequence_length,
        time_major=False,
        is_reverse=True)
    # 拼接前向与反向GRU的编码结果得到h
    encoder_output = layers.concat(
        input=[encoder_fwd_output, encoder_bwd_output], axis=2)
    encoder_state = layers.concat(input=[fwd_state, bwd_state], axis=1)
    return encoder_output, encoder_state

In [4]:
class DecoderCell(layers.RNNCell):
    def __init__(self, hidden_size):
        self.hidden_size = hidden_size
        self.gru_cell = layers.GRUCell(hidden_size)

    def attention(self, hidden, encoder_output, encoder_output_proj,
                  encoder_padding_mask):
        # 定义attention用以计算context，即 c_i，这里使用Bahdanau attention机制
        decoder_state_proj = layers.unsqueeze(
            layers.fc(hidden, size=self.hidden_size, bias_attr=False), [1])
        mixed_state = fluid.layers.elementwise_add(
            encoder_output_proj,
            layers.expand(decoder_state_proj,
                        [1, layers.shape(decoder_state_proj)[1], 1]))
        attn_scores = layers.squeeze(
            layers.fc(input=mixed_state,
                    size=1,
                    num_flatten_dims=2,
                    bias_attr=False), [2])
        if encoder_padding_mask is not None:
            attn_scores = layers.elementwise_add(attn_scores,
                                                encoder_padding_mask)
        attn_scores = layers.softmax(attn_scores)
        context = layers.reduce_sum(layers.elementwise_mul(encoder_output,
                                                        attn_scores,
                                                        axis=0),
                                    dim=1)
        return context

    def call(self,
             step_input,
             hidden,
             encoder_output,
             encoder_output_proj,
             encoder_padding_mask=None):
        # Bahdanau attention
        context = self.attention(hidden, encoder_output, encoder_output_proj,
                                encoder_padding_mask)
        step_input = layers.concat([step_input, context], axis=1)
        # GRU
        output, new_hidden = self.gru_cell(step_input, hidden)
        return output, new_hidden

In [5]:
def decoder(encoder_output,
            encoder_output_proj,
            encoder_state,
            encoder_padding_mask,
            trg=None,
            is_train=True):
    # 定义 RNN 所需要的组件
    decoder_cell = DecoderCell(hidden_size=decoder_size)
    decoder_initial_states = layers.fc(encoder_state,
                                    size=decoder_size,
                                    act="tanh")
    trg_embeder = lambda x: fluid.embedding(input=x,
                                            size=[target_dict_size, hidden_dim],
                                            dtype="float32",
                                            param_attr=fluid.ParamAttr(
                                                name="trg_emb_table"))
    output_layer = lambda x: layers.fc(x,
                                       size=target_dict_size,
                                       num_flatten_dims=len(x.shape) - 1,
                                       param_attr=fluid.ParamAttr(name=
                                                                  "output_w"))
    if is_train:  # 训练
        # 训练时使用 `layers.rnn` 构造由 `cell` 指定的循环神经网络
        # 循环的每一步从 `inputs` 中切片产生输入，并执行 `cell.call`
        decoder_output, _ = layers.rnn(
            cell=decoder_cell,
            inputs=trg_embeder(trg),
            initial_states=decoder_initial_states,
            time_major=False,
            encoder_output=encoder_output,
            encoder_output_proj=encoder_output_proj,
            encoder_padding_mask=encoder_padding_mask)
        decoder_output = output_layer(decoder_output)
    else:  # 基于 beam search 的预测生成
        # beam search 时需要将用到的形为 `[batch_size, ...]` 的张量扩展为 `[batch_size* beam_size, ...]`
        encoder_output = layers.BeamSearchDecoder.tile_beam_merge_with_batch(
            encoder_output, beam_size)
        encoder_output_proj = layers.BeamSearchDecoder.tile_beam_merge_with_batch(
            encoder_output_proj, beam_size)
        encoder_padding_mask = layers.BeamSearchDecoder.tile_beam_merge_with_batch(
            encoder_padding_mask, beam_size)
        # BeamSearchDecoder 定义了单步解码的操作：`cell.call` + `beam_search_step`
        beam_search_decoder = layers.BeamSearchDecoder(cell=decoder_cell,
                                                       start_token=bos_id,
                                                       end_token=eos_id,
                                                       beam_size=beam_size,
                                                       embedding_fn=trg_embeder,
                                                       output_fn=output_layer)
        # 使用 layers.dynamic_decode 动态解码
        # 重复执行 `decoder.step()` 直到其返回的表示完成状态的张量中的值全部为True或解码步骤达到 `max_step_num`
        decoder_output, _ = layers.dynamic_decode(
            decoder=beam_search_decoder,
            inits=decoder_initial_states,
            max_step_num=max_length,
            output_time_major=False,
            encoder_output=encoder_output,
            encoder_output_proj=encoder_output_proj,
            encoder_padding_mask=encoder_padding_mask)

    return decoder_output

In [6]:
def model_func(inputs, is_train=True):
    # 源语言输入
    src = inputs[0]
    src_sequence_length = inputs[1]
    src_embeder = lambda x: fluid.embedding(
        input=x,
        size=[source_dict_size, hidden_dim],
        dtype="float32",
        param_attr=fluid.ParamAttr(name="src_emb_table"))
    src_embedding = src_embeder(src)

    # 编码器
    encoder_output, encoder_state = encoder(src_embedding, src_sequence_length)

    encoder_output_proj = layers.fc(input=encoder_output,
                                    size=decoder_size,
                                    num_flatten_dims=2,
                                    bias_attr=False)
    src_mask = layers.sequence_mask(src_sequence_length,
                                    maxlen=layers.shape(src)[1],
                                    dtype="float32")
    encoder_padding_mask = (src_mask - 1.0) * 1e9

    # 目标语言输入，训练时有、预测生成时无该输入
    trg = inputs[2] if is_train else None

    # 解码器
    output = decoder(encoder_output=encoder_output,
                     encoder_output_proj=encoder_output_proj,
                     encoder_state=encoder_state,
                     encoder_padding_mask=encoder_padding_mask,
                     trg=trg,
                     is_train=is_train)
    return output

In [7]:
def loss_func(logits, label, trg_sequence_length):
    probs = layers.softmax(logits)
    # 使用交叉熵损失函数
    loss = layers.cross_entropy(input=probs, label=label)
    # 根据长度生成掩码，并依此剔除 padding 部分计算的损失
    trg_mask = layers.sequence_mask(trg_sequence_length,
                                    maxlen=layers.shape(logits)[1],
                                    dtype="float32")
    avg_cost = layers.reduce_sum(loss * trg_mask) / layers.reduce_sum(trg_mask)
    return avg_cost

def optimizer_func():
    # 设置梯度裁剪
    fluid.clip.set_gradient_clip(
        clip=fluid.clip.GradientClipByGlobalNorm(clip_norm=5.0))
    # 定义先增后降的学习率策略
    lr_decay = fluid.layers.learning_rate_scheduler.noam_decay(hidden_dim, 1000)
    return fluid.optimizer.Adam(
        learning_rate=lr_decay,
        regularization=fluid.regularizer.L2DecayRegularizer(
            regularization_coeff=1e-4))

In [8]:
def inputs_generator(batch_size, pad_id, is_train=True):
    data_generator = fluid.io.shuffle(
        paddle.dataset.wmt16.train(source_dict_size, target_dict_size),
        buf_size=10000) if is_train else paddle.dataset.wmt16.test(
            source_dict_size, target_dict_size)
    batch_generator = fluid.io.batch(data_generator, batch_size=batch_size)

    # 对 batch 内的数据进行 padding
    def _pad_batch_data(insts, pad_id):
        seq_lengths = np.array(list(map(len, insts)), dtype="int64")
        max_len = max(seq_lengths)
        pad_data = np.array(
            [inst + [pad_id] * (max_len - len(inst)) for inst in insts],
            dtype="int64")
        return pad_data, seq_lengths

    def _generator():
        for batch in batch_generator():
            batch_src = [ins[0] for ins in batch]
            src_data, src_lengths = _pad_batch_data(batch_src, pad_id)
            inputs = [src_data, src_lengths]
            if is_train:  #训练时包含 target 和 label 数据
                batch_trg = [ins[1] for ins in batch]
                trg_data, trg_lengths = _pad_batch_data(batch_trg, pad_id)
                batch_lbl = [ins[2] for ins in batch]
                lbl_data, _ = _pad_batch_data(batch_lbl, pad_id)
                inputs += [trg_data, trg_lengths, lbl_data]
            yield inputs

    return _generator

In [9]:
train_prog = fluid.Program()
startup_prog = fluid.Program()
with fluid.program_guard(train_prog, startup_prog):
    with fluid.unique_name.guard():
        # 训练时：
        # inputs = [src, src_sequence_length, trg, trg_sequence_length, label]
        inputs, loader = data_func(is_train=True)
        logits = model_func(inputs, is_train=True)
        loss = loss_func(logits, inputs[-1], inputs[-2])
        optimizer = optimizer_func()
        optimizer.minimize(loss)

In [13]:
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64/

In [14]:
# 设置训练设备
use_cuda = True
places = fluid.cuda_places() if use_cuda else fluid.cpu_places()
# 设置数据源
loader.set_batch_generator(inputs_generator(batch_size,
                                            eos_id,
                                            is_train=True),
                            places=places)
# 定义执行器，初始化参数并绑定Program
exe = fluid.Executor(places[0])
exe.run(startup_prog)
prog = fluid.CompiledProgram(train_prog).with_data_parallel(
    loss_name=loss.name)

EnforceNotMet: 

--------------------------------------------
C++ Call Stacks (More useful to developers):
--------------------------------------------
0   std::string paddle::platform::GetTraceBackString<char const*>(char const*&&, char const*, int)
1   paddle::platform::EnforceNotMet::EnforceNotMet(std::__exception_ptr::exception_ptr, char const*, int)
2   paddle::platform::dynload::GetCublasDsoHandle()
3   void std::__once_call_impl<std::_Bind_simple<paddle::platform::dynload::DynLoad__cublasCreate_v2::operator()<cublasContext**>(cublasContext**)::{lambda()#1} ()> >()
4   paddle::platform::CublasHandleHolder::CublasHandleHolder(CUstream_st*, cublasMath_t)
5   paddle::platform::CUDADeviceContext::CUDADeviceContext(paddle::platform::CUDAPlace)
6   std::_Function_handler<std::unique_ptr<paddle::platform::DeviceContext, std::default_delete<paddle::platform::DeviceContext> > (), std::reference_wrapper<std::_Bind_simple<paddle::platform::EmplaceDeviceContext<paddle::platform::CUDADeviceCo

In [19]:
EPOCH_NUM = 2
for pass_id in six.moves.xrange(EPOCH_NUM):
    batch_id = 0
    for data in loader():
        loss_val = exe.run(prog, feed=data, fetch_list=[loss])[0]
        print('pass_id: %d, batch_id: %d, loss: %f' %
                (pass_id, batch_id, loss_val))
        batch_id += 1
    # 保存模型
    fluid.io.save_params(exe, model_save_dir, main_program=train_prog)

EnforceNotMet: 

--------------------------------------------
C++ Call Stacks (More useful to developers):
--------------------------------------------
0   std::string paddle::platform::GetTraceBackString<char const*>(char const*&&, char const*, int)
1   paddle::platform::EnforceNotMet::EnforceNotMet(std::__exception_ptr::exception_ptr, char const*, int)
2   paddle::platform::dynload::GetCublasDsoHandle()
3   void std::__once_call_impl<std::_Bind_simple<paddle::platform::dynload::DynLoad__cublasCreate_v2::operator()<cublasContext**>(cublasContext**)::{lambda()#1} ()> >()
4   paddle::platform::CublasHandleHolder::CublasHandleHolder(CUstream_st*, cublasMath_t)
5   paddle::platform::CUDADeviceContext::CUDADeviceContext(paddle::platform::CUDAPlace)
6   std::_Function_handler<std::unique_ptr<paddle::platform::DeviceContext, std::default_delete<paddle::platform::DeviceContext> > (), std::reference_wrapper<std::_Bind_simple<paddle::platform::EmplaceDeviceContext<paddle::platform::CUDADeviceCo

In [56]:
gru = fluid.layers.GRUCell(hidden_size=256)
output, state = fluid.layers.rnn(cell=gru, 
                                        inputs=example_input_batch)

ValueError: Data Type mismatch: 2 to 5

In [53]:
encoder = Encoder("Encoder", vocab_size, embedding_dim, embedding_matrix, units, BATCH_SIZE)
# example_input
example_input_batch = fluid.layers.ones(shape=(BATCH_SIZE,max_length_inp), dtype="int32")
# sample input
#sample_hidden = encoder.initialize_hidden_state()


In [37]:
sample_hidden
import pdb; 

In [54]:
sample_output, sample_hidden = encoder(example_input_batch)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

NotImplementedError: 

In [ ]:
with_gpu=True
print "with gpu: ", with_gpu

def save_model(trainer, save_path):
    with open(save_path, 'w') as f:
        trainer.save_parameter_to_tar(f)

def load_word2vec_file(word2vec_file):
    word2vec_dict = {}
    input = codecs.open(word2vec_file, "r", "utf-8")
    lines = input.readlines()
    input.close()
    word_num, dim = lines[0].split(" ")
    word_num = int(word_num)
    dim = int(dim)
    
    lines = lines[1:]
    for l in lines:
        l = l.strip()
        tokens = l.split(" ")
        if len(tokens) != dim + 1:
            continue
        w = tokens[0]
        v = np.array(map(lambda x:float(x), tokens[1:]))
        word2vec_dict[w] = v
    return word2vec_dict, dim


def seq_to_seq_net(vocab_size,
                   is_generating,
                   word_vector_dim=100,
                   hidden_size=100,
                   beam_size=3,
                   max_length=100,
                   dropout_rate=0.):

    # Network Architecture
    decoder_size = hidden_size  # dimension of hidden unit of GRU decoder
    encoder_size = hidden_size  # dimension of hidden unit of GRU encoder

    embedding_param = paddle.attr.ParamAttr(name='embedding')

    def embedding_layer(word_id):
        embed = paddle.layer.embedding(
            input=word_id, size=word_vector_dim, param_attr=embedding_param)
        return paddle.layer.dropout(input=embed, dropout_rate=dropout_rate)


    def BiGru_layer(embedding):
        forward = paddle.networks.simple_gru2(
            input=embedding, size=encoder_size, 
            gru_param_attr=paddle.attr.ParamAttr(name='gru_forward_encoder'),
            #gru_bias_attr=False,
            #mixed_bias_attr=False,
            mixed_param_attr=paddle.attr.ParamAttr(name='mixed_forward_encoder'))
            
        backward = paddle.networks.simple_gru2(
            input=embedding, size=encoder_size, reverse=True,
            gru_param_attr=paddle.attr.ParamAttr(name='gru_backward_encoder'),
            #gru_bias_attr=False,
            #mixed_bias_attr=False,
            mixed_param_attr=paddle.attr.ParamAttr(name='mixed_backward_encoder'))
        return forward, backward


    #### Encoder
    ##### Encode problem to a fixed size vector
    problem_word_id = paddle.layer.data(
        name='problem_word',
        type=paddle.data_type.integer_value_sequence(vocab_size))

    problem_embedding = embedding_layer(problem_word_id)

    problem_f, problem_b = BiGru_layer(problem_embedding)
    problem_f_last = paddle.layer.last_seq(input=problem_f)
    problem_b_first = paddle.layer.first_seq(input=problem_b)

    problem_vector = paddle.layer.concat(input=[problem_f_last, problem_b_first])
    problem_vector = paddle.layer.dropout(input=problem_vector, dropout_rate=dropout_rate)

    ##### Encode conversation
    conversation_word_id = paddle.layer.data(
        name='conversation_word',
        type=paddle.data_type.integer_value_sequence(vocab_size))

    conversation_embedding = embedding_layer(conversation_word_id)
    conversation_f, conversation_b = BiGru_layer(conversation_embedding)
    conversation_vector = paddle.layer.concat(input=[conversation_f, conversation_b])

    #### Decoder
    conversation_proj = paddle.layer.fc(
        act=paddle.activation.Linear(),
        size=decoder_size,
        bias_attr=False,
        input=conversation_vector)

    backward_first = paddle.layer.first_seq(input=conversation_b)

    decoder_boot = paddle.layer.fc(
        size=decoder_size,
        act=paddle.activation.Tanh(),
        bias_attr=False,
        input=backward_first)

    def gru_decoder_with_attention(
            enc_vec, enc_proj, problem_vec, current_word):

        decoder_mem = paddle.layer.memory(
            name='gru_decoder', size=decoder_size, boot_layer=decoder_boot)

        context = paddle.networks.simple_attention(
            encoded_sequence=enc_vec,
            encoded_proj=enc_proj,
            decoder_state=decoder_mem)

        decoder_inputs = paddle.layer.fc(
            act=paddle.activation.Linear(),
            size=decoder_size * 3,
            bias_attr=False,
            input=[context, problem_vec, current_word],
            layer_attr=paddle.attr.ExtraLayerAttribute(
                error_clipping_threshold=100.0))

        gru_step = paddle.layer.gru_step(
            name='gru_decoder',
            input=decoder_inputs,
            output_mem=decoder_mem,
            size=decoder_size)
        
        out = paddle.layer.mixed(size=vocab_size,
            act=paddle.activation.Softmax(),
            bias_attr=False,
            ## 这里使用 embedding 矩阵参数 产生 report 的词
            input=paddle.layer.trans_full_matrix_projection(
                        input=gru_step,
                        param_attr=embedding_param))
        
        return out

    decoder_group_name = 'decoder_group'
    group_input1 = paddle.layer.StaticInput(input=conversation_vector)
    group_input2 = paddle.layer.StaticInput(input=conversation_proj)
    group_input3 = paddle.layer.StaticInput(input=problem_vector)
    group_inputs = [group_input1, group_input2, group_input3]

    if not is_generating:
        report_embedding = embedding_layer(paddle.layer.data(
            name='report_word',
            type=paddle.data_type.integer_value_sequence(vocab_size)))

        group_inputs.append(report_embedding)

        decoder = paddle.layer.recurrent_group(
            name=decoder_group_name,
            step=gru_decoder_with_attention,
            input=group_inputs)

        lbl = paddle.layer.data(
            name='report_next_word',
            type=paddle.data_type.integer_value_sequence(vocab_size))
        cost = paddle.layer.classification_cost(input=decoder, label=lbl)

        return cost
    else:
        report_embedding = paddle.layer.GeneratedInput(
            size=vocab_size,
            embedding_name='embedding',
            embedding_size=word_vector_dim)
        group_inputs.append(report_embedding)

        beam_gen = paddle.layer.beam_search(
            name=decoder_group_name,
            step=gru_decoder_with_attention,
            input=group_inputs,
            bos_id=0,
            eos_id=1,
            beam_size=beam_size,
            max_length=max_length)

        return beam_gen

In [21]:
embedding_matrix_pa()

TypeError: __call__() missing 2 required positional arguments: 'var' and 'block'

In [13]:
embedding_matrix_pa = paddle.fluid.initializer.NumpyArrayInitializer(embedding_matrix)

In [15]:
w_param_attrs = fluid.ParamAttr(name="fc_weight",
                                trainable=False)

In [16]:
y_predict = fluid.layers.fc(input=embedding_matrix_pa, size=10, param_attr=w_param_attrs)

TypeError: The type of 'input' in fc must be Variable, but received <class 'paddle.fluid.initializer.NumpyArrayInitializer'>

In [22]:
embedding_matrix.shape

(33064, 200)

In [23]:
x = fluid.layers.data(name='X', shape=embedding_matrix.shape, dtype='float32')

In [24]:
embedding_matrix_pa(x, )

TypeError: __call__() missing 1 required positional argument: 'block'

In [25]:
x = fluid.layers.data(name="x", shape=[33064], dtype='float32')
fc = fluid.layers.fc(input=x, size=6612800,
    param_attr=fluid.initializer.NumpyArrayInitializer(embedding_matrix))

In [28]:
w_param_attrs = fluid.ParamAttr(
    name="emb_weight",
    initializer=fluid.initializer.NumpyArrayInitializer(embedding_matrix),
    trainable=False)

In [29]:
emb = fluid.dygraph.Embedding(
        name_scope='embedding',
        size=[33064, 200],
        param_attr= w_param_attrs,
        is_sparse=False)

In [31]:
import numpy as np
inp_word = np.array([[[1]]]).astype('int64')

In [32]:
import paddle.fluid.dygraph.base as base
emb(base.to_variable(inp_word))

AssertionError: We Only support to_variable in Dygraph mode, please use fluid.dygraph.guard() as context to run it in Dygraph Mode